In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('final.csv')
out_of_sample = pd.read_csv('Out_of_Sample.csv')

In [2]:
x = df.loc[:,'Visitor-Points':'elo_diff'].values
x.shape

(5967, 136)

In [3]:
y = df.loc[:,['score_gap_sign']].values

In [4]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=1/7.0, random_state=0)

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_x)
train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

In [6]:
train_y = train_y.ravel()

In [7]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVR(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

svc_param_selection(train_x, train_y, 5)

KeyboardInterrupt: 

In [ ]:
from sklearn.svm import SVR
svregress = SVR(kernel='rbf', C=10, gamma=0.001)
svregress.fit(train_x, train_y)  

In [ ]:
pred = svregress.predict(test_x)

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

error = np.sqrt(mean_squared_error(test_y, pred))
print(error)

In [ ]:
# out-of-sample prediction
X_out = out_of_sample.drop(['score_gap_sign'], axis = 1)
y_out = out_of_sample['score_gap_sign']
scaler = StandardScaler()
scaler.fit(X_out)
X_out = scaler.transform(X_out)
svregress = SVR(kernel='rbf', C=10, gamma=0.001)
svregress.fit(x,y)
pred = svregress.predict(X_out)
error = np.sqrt(mean_squared_error(y_out, pred))
print(error)

In [ ]:
print(pred)

In [ ]:
# Visulization of prediction 
import matplotlib.pyplot as plt
pred_list = list(pred)
y_out_list = list(y_out)
fig, ax = plt.subplots()
plt.xlabel('games')
plt.ylabel('actual score gap; predicted score gap')
yticks = range(-40,40,5)
ax.set_yticks(yticks)
ax.set_ylim([-40,40])
x = list(range(len(out_of_sample)))
plt.plot(x,pred_list,"x-",label="predicted score gap")
plt.plot(x,y_out_list,"+-",label="actual score gap")
plt.grid(True)
plt.legend(bbox_to_anchor=(1.0, 1), loc=1, borderaxespad=0.)
